In [2]:
from manim import *
from numpy import linalg as npl
import matplotlib.pyplot as plt
import jupyter_capture_output
from scipy.interpolate import make_interp_spline, BSpline

video_scene = " -v WARNING --progress_bar None --disable_caching mfor_Scene"
image_scene = f" -v WARNING --progress_bar None --disable_caching -r {2*427},{2*240}  -s mfor_Scene"

Manim Community v0.16.0.post0

Jupyter Capture Output v0.0.8


In [126]:
# Trajektorie der Parabel: a*(x - x_shift)**2 + b
def traj_ball(t):  
    return (4*t - 16, 10*t - 2 * t**2, 0)


# Trajektorie des bewegten reference-frames: a*x + b
def traj_mfor(t):
    x = -2*t
    a = -7.5/10
    return (x, a*x, 0)

In [150]:
%%capture_video --path "animations/moved_frame_of_reference/moved_frame_of_reference.mp4"
%%manim -qm --fps 20 $video_scene

class mfor_Scene(Scene):
    def construct(self):
        CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
        self.add(CVC)

        text1 = Text("(a) Ruhendes Bezugssystem", font_size = 36).align_on_border(UP + LEFT, buff = 0.5)
        text2 = Text("(b) Bewegtes Bezugssystem", font_size = 36).align_on_border(UP + LEFT, buff = 0.5)

        npla = NumberPlane(
            x_range = [-50, 50, 2], y_range  = [-50, 50, 2], x_length = 50, y_length = 40, 
            axis_config = {"include_numbers": False, "stroke_opacity": 0.25}, background_line_style = {"stroke_opacity": 0.25}).move_to([3, -2, -1])        

        mfor_obj1 = ImageMobject("media/images/manim_animations/carlo_close_template.jpg").move_to(npla.c2p(traj_mfor(0)[0], traj_mfor(0)[1], 0))
        ball = Circle(radius = 0.125, color = RED, fill_opacity = 0.5).move_to(npla.c2p(traj_ball(0)[0], traj_ball(0)[1], 0))

        mfor_x_axis = Line(npla.c2p(-50, traj_mfor(0)[1], 0), npla.c2p(50, traj_mfor(0)[1], 0), color = WHITE, stroke_width = 0.5)
        mfor_y_axis = Line(npla.c2p(traj_mfor(0)[0], -50, 0), npla.c2p(traj_mfor(0)[0], 50, 0), color = WHITE, stroke_width = 0.5)

        mfor_x = Text("x'", font_size = 16).move_to([6.75, npla.c2p(0, traj_mfor(0)[1] + 0.5, 0)[1], 0])
        mfor_y = Text("y'", font_size = 16).move_to([npla.c2p(traj_mfor(0)[0] + 0.5, 0, 0)[0], 3.75, 0])
        
        
        def ball_updater(obj):
            t = time.get_value()
            x = traj_ball(t)[0]
            y = traj_ball(t)[1]
            obj.move_to(npla.c2p(x, y, 0))


        def mfor_updater(obj):
            t = time.get_value()
            x = traj_mfor(t)[0]
            y = traj_mfor(t)[1]
            obj.move_to(npla.c2p(x, y, 0))

        def mfor_x_axis_updater(axis):
            t = time.get_value()
            y = traj_mfor(t)[1]
            axis.move_to(npla.c2p(0, y, 0))

        def mfor_y_axis_updater(axis):
            t = time.get_value()
            x = traj_mfor(t)[0]
            axis.move_to(npla.c2p(x, 0, 0))

        def mfor_x_updater(label):
            t = time.get_value()
            y = npla.c2p(0, traj_mfor(t)[1] + 0.5, 0)[1]
            label.move_to([6.75, y, 0])

        def mfor_y_updater(label):
            t = time.get_value()
            x = npla.c2p(traj_mfor(t)[0] + 0.5, 0, 0)[0]
            label.move_to([x, 3.75, 0])

        self.add(npla)
        self.play(Write(text1), run_time = 1.5)
        self.play(FadeIn(mfor_obj1), FadeIn(ball), FadeIn(mfor_x_axis), FadeIn(mfor_y_axis), FadeIn(mfor_x), FadeIn(mfor_y), run_time = 3)
        self.play(Unwrite(text1), run_time = 1.5)
        self.wait(0.5)
        
        self.add(mfor_obj1)
        self.add(ball)

        self.add(mfor_x_axis, mfor_y_axis)
        self.add(mfor_x, mfor_y)
        

        time = ValueTracker(0)

        ball.add_updater(ball_updater)
        mfor_obj1.add_updater(mfor_updater)

        mfor_x_axis.add_updater(mfor_x_axis_updater)
        mfor_y_axis.add_updater(mfor_y_axis_updater)

        mfor_x.add_updater(mfor_x_updater)
        mfor_y.add_updater(mfor_y_updater)

        self.play(time.animate.set_value(7.5), rate_func = linear, run_time = 7.5)
        self.wait(1.5)
        self.play(FadeOut(mfor_obj1), FadeOut(ball), FadeOut(mfor_x_axis), FadeOut(mfor_y_axis), FadeOut(mfor_x), FadeOut(mfor_y), run_time = 3)
        self.play(Write(text2), run_time = 1.5)


Output saved by overwring previous file at animations/moved_frame_of_reference/moved_frame_of_reference.mp4.
